\
Реализуем метод Монте-Карло напрямую, моделируя броуновское движение, количество скачков и их размер.

Используем геометрическое броуновское движение, заданное следующей формулой:
$$
d S_t= \mu S_t d t+\sigma S_t d W_t
$$

Скачки определяются процессом Пуассона:
$$
d S_t= J S_t  d N_t
$$

где:
- $J$ - размер скачка, определяется распределением:
$$
J_+ \sim exp\left(\frac{1}{\Lambda_+}\right), \quad
J_- \sim exp\left(\frac{1}{\Lambda_-}\right)
$$
Скачки выбираются со следующими вероятностями:
$$
P\left(J = J_+\right) = p, \quad
P\left(J = J_-\right) = 1 - p
$$

- $dN_t$ - это количество скачков, которые происходят в течение небольшого временного интервала $dt$, описываемое пуассоновским процессом с интенсивностью $\lambda=\lambda_{+}+\lambda_{-}$.

Формула, объединяющая броуновское движение и скачки:

$$
d S_t=\mu S_t d t+\sigma S_t d W_t + J S_t d N_t
$$

Или же:

$$
S_t=S_{t-1} \cdot e^{ \left(\mu d t+\sigma d W_t+J d N_t\right)},
$$
где параметр смещения может быть вычислен по следующей формуле:

$$
\mu = r - \frac{1}{2} \sigma^2 - \frac{p \cdot \lambda}{\Lambda_+ - 1} + \frac{(1 - p) \cdot \lambda}{\Lambda_- + 1}
$$


In [17]:
def compute_mu(r, sigma, intensity, lambda_p, lambda_m, p):
    kappa_plus = (p * intensity) / (lambda_p - 1)
    kappa_minus = -1 * (1 - p) * intensity / (lambda_m + 1)
    kappa = kappa_plus + kappa_minus
    mu = r - 0.5 * sigma ** 2 - kappa
    return mu

In [21]:
def simulate_jump_diffusion(S0, r, sigma, T, intensity, lambda_p, lambda_m, p, M, N):
    dt = T / N
    mu = compute_mu(r, sigma, intensity, lambda_p, lambda_m, p)
    S = np.full(M, S0, dtype=float)

    for _ in range(N):
        dW = np.random.normal(0, np.sqrt(dt), M)
        N_jumps = np.random.poisson((lambda_p + lambda_m) * dt, M)

        jump_directions = np.random.rand(M) < p
        pos_jumps = np.random.exponential(1 / lambda_p, M)
        neg_jumps = -np.random.exponential(1 / lambda_m, M)
        jumps = np.where(jump_directions, pos_jumps, neg_jumps) * N_jumps

        S *= np.exp(mu * dt + sigma * dW + jumps)

    return S

\
Cтоимость колл-опциона будем рассчитывать по следующей формуле:

$$C=e^{-r T} \cdot \mathbb{E}\left[\max \left(S_T-K, 0\right)\right],$$

где
$$\mathbb{E}\left[\max \left(S_T-K, 0\right)\right] = e^{-rT}\frac{1}{M} \sum_{i=1}^{M} max(S_T-K, 0)$$

Ошибку метода можно вычислить по следующей формуле:
$$
\delta=t(1-\gamma, n-1) \sqrt{\frac{S^2}{n}},
$$
где
- n - количество симуляций,
- $
S^2 = \frac{1}{M-1} \sum_{i=1}^M\left(\text { payoff }_i-{\overline{\text { payoff }})^2}\right.
$
- $t(1-\gamma, n - 1)$ - критическое значения t-распределения Стьюдента, причём $t(1-\gamma, n - 1)\approx t(\gamma)$ - для больших n

In [19]:

def price_option_with_error(S, K, r, T, confidence=0.95):
    discounted_payoffs = np.exp(-r * T) * np.maximum(S - K, 0)
    mean = np.mean(discounted_payoffs)
    std = np.std(discounted_payoffs, ddof=1)
    t_value = t.ppf(confidence, len(S) - 1)
    error = t_value * std / np.sqrt(len(S))
    return mean, error

\
Пусть имеется набор параметров $c_+ = 1$, $c_- = 1$, $\lambda_+ = 10$, $\lambda_- = -15$.

Тогда, $\Lambda_+ = -\lambda_- = 15, \Lambda_- = \lambda_+ = 10$

Также $c_+ = (1-p)\lambda,$ и $ c_- = pλ$

Тогда $\frac{c_+}{(1-p)} = \frac{c_-}{p} \Longrightarrow p = 0.5$

In [20]:
import numpy as np
from scipy.stats import t

S0, K, r, q, T, sigma = 100, 80, 0.05, 0, 10/252, 0.3
intensity, lambda_p, lambda_m, p = 12, 15, 10, 0.5
M, N = 10000, 256

S_T = simulate_jump_diffusion(S0, r, sigma, T, intensity, lambda_p, lambda_m, p, M, N)
price, error = price_option_with_error(S_T, K, r, T)

print(f"Цена опциона: {price:.4f}")
print(f"Абсолютная ошибка: {error:.4f}")
print(f"Относительная ошибка: {error/price:.4f}")


0.12188311688311687
Цена опциона: 20.2113
Абсолютная ошибка: 0.1938
Относительная ошибка: 0.0096
